In [1]:
%matplotlib inline

Gensim Tutorial
=============
gensim: https://radimrehurek.com/gensim/

gensim API: https://radimrehurek.com/gensim/apiref.html#api-reference


Core Concepts
=============


This tutorial introduces Documents, Corpora, Vectors and Models: the basic concepts and terms needed to understand and use gensim.



In [2]:
import pprint

The core concepts of ``gensim`` are:

1. `core_concepts_document`: some text.
2. `core_concepts_corpus`: a collection of documents.
3. `core_concepts_vector`: a mathematically convenient representation of a document.
4. `core_concepts_model`: an algorithm for transforming vectors from one representation to another.

Let's examine each of these in slightly more detail.


Document
--------

In Gensim, a *document* is an object of the [text sequence type](https://docs.python.org/3.7/library/stdtypes.html#text-sequence-type-str)_ (commonly known as ``str`` in Python 3).
A document could be anything from a short 280 character tweet, a single
paragraph (i.e., journal article abstract), a news article, or a book.




In [3]:
document = "Human machine interface for lab abc computer applications"


Corpus
------

A *corpus* is a collection of `core_concepts_document` objects.
Corpora serve two roles in Gensim:

1. Input for training a `core_concepts_model`.
   During training, the models use this *training corpus* to look for common
   themes and topics, initializing their internal model parameters.

   Gensim focuses on *unsupervised* models so that no human intervention,
   such as costly annotations or tagging documents by hand, is required.

2. Documents to organize.
   After training, a topic model can be used to extract topics from new
   documents (documents not seen in the training corpus).

   Such corpora can be indexed for [Similarity Queries](https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html#sphx-glr-auto-examples-core-run-similarity-queries-py)
   queried by semantic similarity, clustered etc.

Here is an example corpus.
It consists of 9 documents, where each document is a string consisting of a single sentence.




In [4]:
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]
# nine documents: each documents is actually a sentence. 


The above example loads the entire corpus into memory.
In practice, corpora may be very large, so loading them into memory may be impossible.
Gensim intelligently handles such corpora by *streaming* them one document at a time.
See [Corpus Streaming – One Document at a Time](https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html#corpus-streaming-tutorial) for details.

This is a particularly small example of a corpus for illustration purposes.
Another example could be a list of all the plays written by Shakespeare, list
of all wikipedia articles, or all tweets by a particular person of interest.

After collecting our corpus, there are typically a number of preprocessing
steps we want to undertake. We'll keep it simple and just remove some
commonly used English words (such as 'the') and words that occur only once in
the corpus. In the process of doing so, we'll tokenize our data.
Tokenization breaks up the documents into words (in this case using space as
a delimiter).


There are better ways to perform preprocessing than just lower-casing and
splitting by space.  Effective preprocessing is beyond the scope of this
tutorial: if you're interested, check out the
[gensim.utils.simple_preprocess()](https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess) function.




In [5]:
# Preprocessing

# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))

# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


Before proceeding, we want to associate each word in the corpus with a unique
integer ID. We can do this using the `gensim.corpora.Dictionary`
class.  This dictionary defines the vocabulary of all words that our
processing knows about.




In [9]:
pip install gensim

  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda list

In [10]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

# reason we bulid dictionary: identify total number of words, which we say vocabularly

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


Because our corpus is small, there are only 12 different tokens in this
`gensim.corpora.Dictionary`. For larger corpuses, dictionaries that
contains hundreds of thousands of tokens are quite common.





Vector
------

To infer the latent structure in our corpus we need a way to represent
documents that we can manipulate mathematically. One approach is to represent
each document as a vector of *features*.
For example, a single feature may be thought of as a question-answer pair:

1. How many times does the word *splonge* appear in the document? Zero.
2. How many paragraphs does the document consist of? Two.
3. How many fonts does the document use? Five.

The question is usually represented only by its integer id (such as `1`, `2` and `3`).
The representation of this document then becomes a series of pairs like ``(1, 0.0), (2, 2.0), (3, 5.0)``.
This is known as a *dense vector*, because it contains an explicit answer to each of the above questions.

If we know all the questions in advance, we may leave them implicit
and simply represent the document as ``(0, 2, 5)``.
This sequence of answers is the **vector** for our document (in this case a 3-dimensional dense vector).
For practical purposes, only questions to which the answer is (or
can be converted to) a *single floating point number* are allowed in Gensim.

In practice, vectors often consist of many zero values.
To save memory, Gensim omits all vector elements with value 0.0.
The above example thus becomes ``(2, 2.0), (3, 5.0)``.
This is known as a *sparse vector* or *bag-of-words vector*.
The values of all missing features in this sparse representation can be unambiguously resolved to zero, ``0.0``.

Assuming the questions are the same, we can compare the vectors of two different documents to each other.
For example, assume we are given two vectors ``(0.0, 2.0, 5.0)`` and ``(0.1, 1.9, 4.9)``.
Because the vectors are very similar to each other, we can conclude that the documents corresponding to those vectors are similar, too.
Of course, the correctness of that conclusion depends on how well we picked the questions in the first place.

Another approach to represent a document as a vector is the *bag-of-words
model*.
Under the bag-of-words model each document is represented by a vector
containing the frequency counts of each word in the dictionary.
For example, assume we have a dictionary containing the words
``['coffee', 'milk', 'sugar', 'spoon']``.
A document consisting of the string ``"coffee milk coffee"`` would then
be represented by the vector ``[2, 1, 0, 0]`` where the entries of the vector
are (in order) the occurrences of "coffee", "milk", "sugar" and "spoon" in
the document. The length of the vector is the number of entries in the
dictionary. One of the main properties of the bag-of-words model is that it
completely ignores the order of the tokens in the document that is encoded,
which is where the name bag-of-words comes from.

Our processed corpus has 12 unique words in it, which means that each
document will be represented by a 12-dimensional vector under the
bag-of-words model. 

We can use the dictionary to turn tokenized documents
into these 12-dimensional vectors. We can see what these IDs correspond to:




In [ ]:
pprint.pprint(dictionary.token2id)

For example, suppose we wanted to vectorize the phrase "Human computer
interaction" (note that this phrase was not in our original corpus). We can
create the bag-of-word representation for a document using the ``doc2bow``
method of the dictionary, which returns a sparse representation of the word
counts:




In [ ]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

The first entry in each tuple corresponds to the ID of the token in the
dictionary, the second corresponds to the count of this token.

Note that "interaction" did not occur in the original corpus and so it was
not included in the vectorization. Also note that this vector only contains
entries for words that actually appeared in the document. Because any given
document will only contain a few words out of the many words in the
dictionary, words that do not appear in the vectorization are represented as
implicitly zero as a space saving measure.

We can convert our entire original corpus to a list of vectors:




In [ ]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

Note that while this list lives entirely in memory, in most applications you
will want a more scalable solution. Luckily, ``gensim`` allows you to use any
iterator that returns a single document vector at a time. See the
documentation for more details.


The distinction between a document and a vector is that the former is text,
and the latter is a mathematically convenient representation of the text.
Sometimes, people will use the terms interchangeably: for example, given
some arbitrary document ``D``, instead of saying "the vector that
corresponds to document ``D``", they will just say "the vector ``D``" or
the "document ``D``".  This achieves brevity at the cost of ambiguity.

As long as you remember that documents exist in document space, and that
vectors exist in vector space, the above ambiguity is acceptable.


Depending on how the representation was obtained, two different documents
may have the same vector representations.


Model
-----

Now that we have vectorized our corpus we can begin to transform it using
*models*. We use model as an abstract term referring to a *transformation* from
one document representation to another. In ``gensim`` documents are
represented as vectors so a model can be thought of as a transformation
between two vector spaces. The model learns the details of this
transformation during training, when it reads the training
`core_concepts_corpus`.

One simple example of a model is [tf-idf](
<https://en.wikipedia.org/wiki/Tf%E2%80%93idf>).  The tf-idf model
transforms vectors from the bag-of-words representation to a vector space
where the frequency counts are weighted according to the relative rarity of
each word in the corpus.

Here's a simple example. Let's initialize the tf-idf model, training it on
our corpus and transforming the string "system minors":




In [ ]:
from gensim import models

# train the model
tfidf = models.TfidfModel(bow_corpus)

# transform the "system minors" string
words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

The ``tfidf`` model again returns a list of tuples, where the first entry is
the token ID and the second entry is the tf-idf weighting. Note that the ID
corresponding to "system" (which occurred 4 times in the original corpus) has
been weighted lower than the ID corresponding to "minors" (which only
occurred twice).

You can save trained models to disk and later load them back, either to
continue training on new training documents or to transform new documents.

``gensim`` offers a number of different models/transformations.
For more, see [Topics and Transformations](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py).

Once you've created the model, you can do all sorts of cool stuff with it.
For example, to transform the whole corpus via TfIdf and index it, in
preparation for similarity queries:




In [ ]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)

and to query the similarity of our query document ``query_document`` against every document in the corpus:



In [ ]:
query_document = 'system engineering'.split()
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

How to read this output?
Document 3 has a similarity score of 0.718=72%, document 2 has a similarity score of 42% etc.
We can make this slightly more readable by sorting:



In [ ]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)


# Corpora and Vector Spaces

Demonstrates transforming text into a vector space representation.

Also introduces corpus streaming and persistence to disk in various formats.


In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

First, let’s create a small corpus of nine short documents.


## From Strings to Vectors



This is a tiny corpus of nine documents, each consisting of only a single sentence.


In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_corpus)
dictionary.save('file/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

Here we assigned a unique integer id to all words appearing in the corpus with the
`gensim.corpora.dictionary.Dictionary` class. This sweeps across the texts, collecting word counts
and relevant statistics. In the end, we see there are twelve distinct words in the
processed corpus, which means each document will be represented by twelve numbers (ie., by a 12-D vector).
To see the mapping between words and their ids:



In [ ]:
print(dictionary.token2id)

To actually convert tokenized documents to vectors:



In [ ]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

The function `doc2bow` simply counts the number of occurrences of
each distinct word, converts the word to its integer word id
and returns the result as a sparse vector. The sparse vector ``[(0, 1), (1, 1)]``
therefore reads: in the document `"Human computer interaction"`, the words `computer`
(id 0) and `human` (id 1) appear once; the other ten dictionary words appear (implicitly) zero times.



In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('file/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

By now it should be clear that the vector feature with ``id=10`` stands for the question "How many
times does the word `graph` appear in the document?" and that the answer is "zero" for
the first six documents and "one" for the remaining three.


## Corpus Streaming -- One Document at a Time

Note that `corpus` above resides fully in memory, as a plain Python list.
In this simple example, it doesn't matter much, but just to make things clear,
let's assume there are millions of documents in the corpus. Storing all of them in RAM won't do.
Instead, let's assume the documents are stored in a file on disk, one document per line. Gensim
only requires that a corpus must be able to return one document vector at a time:




In [ ]:
from smart_open import open  # for transparently opening remote files


class MyCorpus:
    def __iter__(self):
        for line in open('https://radimrehurek.com/mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

The full power of Gensim comes from the fact that a corpus doesn't have to be
a ``list``, or a ``NumPy`` array, or a ``Pandas`` dataframe, or whatever.
Gensim *accepts any object that, when iterated over, successively yields
documents*.



This flexibility allows you to create your own corpus classes that stream the
documents directly from disk, network, database, dataframes... The models
in Gensim are implemented such that they don't require all vectors to reside
in RAM at once. You can even create the documents on the fly!

Download the sample [mycorpus.txt](https://radimrehurek.com/mycorpus.txt) file. The assumption that
each document occupies one line in a single file is not important; you can mold
the `__iter__` function to fit your input format, whatever it is.
Walking directories, parsing XML, accessing the network...
Just parse your input to retrieve a clean list of tokens in each document,
then convert the tokens via a dictionary to their ids and yield the resulting sparse vector inside `__iter__`.



In [ ]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

Corpus is now an object. We didn't define any way to print it, so `print` just outputs address
of the object in memory. Not very useful. To see the constituent vectors, let's
iterate over the corpus and print each document vector (one at a time):



In [ ]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

Although the output is the same as for the plain Python list, the corpus is now much
more memory friendly, because at most one vector resides in RAM at a time. Your
corpus can now be as large as you want.

Similarly, to construct the dictionary without loading all texts into memory:



In [ ]:
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('https://radimrehurek.com/mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)

And that is all there is to it! At least as far as bag-of-words representation is concerned.
Of course, what we do with such a corpus is another question; it is not at all clear
how counting the frequency of distinct words could be useful. As it turns out, it isn't, and
we will need to apply a transformation on this simple representation first, before
we can use it to compute any meaningful document vs. document similarities.

## Corpus Formats

There exist several file formats for serializing a Vector Space corpus (~sequence of vectors) to disk.
`Gensim` implements them via the *streaming corpus interface* mentioned earlier:
documents are read from (resp. stored to) disk in a lazy fashion, one document at
a time, without the whole corpus being read into main memory at once.

One of the more notable file formats is the [Market Matrix format](http://math.nist.gov/MatrixMarket/formats.html).
To save a corpus in the Matrix Market format:

create a toy corpus of 2 documents, as a plain Python list



In [ ]:
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize('file/corpus.mm', corpus)

Other formats include [Joachim's SVMlight format](http://svmlight.joachims.org),
[Blei's LDA-C format](https://radimrehurek.com/gensim/corpora/bleicorpus.html), and
[GibbsLDA++ format](http://gibbslda.sourceforge.net/).



In [ ]:
corpora.SvmLightCorpus.serialize('file/corpus.svmlight', corpus)
corpora.BleiCorpus.serialize('file/corpus.lda-c', corpus)
corpora.LowCorpus.serialize('file/corpus.low', corpus)

Conversely, to load a corpus iterator from a Matrix Market file:



In [ ]:
corpus = corpora.MmCorpus('file/corpus.mm')

Corpus objects are streams, so typically you won't be able to print them directly:



In [ ]:
print(corpus)

Instead, to view the contents of a corpus:



In [ ]:
# one way of printing a corpus: load it entirely into memory
print(list(corpus))  # calling list() will convert any sequence to a plain Python list

or



In [ ]:
# another way of doing it: print one document at a time, making use of the streaming interface
for doc in corpus:
    print(doc)

The second way is obviously more memory-friendly, but for testing and development
purposes, nothing beats the simplicity of calling ``list(corpus)``.

To save the same Matrix Market document stream in Blei's LDA-C format,



In [ ]:
corpora.BleiCorpus.serialize('file/corpus.lda-c', corpus)

In this way, `gensim` can also be used as a memory-efficient **I/O format conversion tool**:
just load a document stream using one format and immediately save it in another format.
Adding new formats is dead easy, check out the [code for the SVMlight corpus](https://github.com/piskvorky/gensim/blob/develop/gensim/corpora/svmlightcorpus.py) for an example.

## Compatibility with NumPy and SciPy

Gensim also contains [efficient utility functions](http://radimrehurek.com/gensim/matutils.html)
to help converting from/to numpy matrices



In [ ]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5, 2])  # random matrix as an example
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
# numpy_matrix = gensim.matutils.corpus2dense(corpus, num_terms=number_of_corpus_features)

and from/to `scipy.sparse` matrices



In [ ]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5, 2)  # random sparse matrix as example
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
#scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)


Topics and Transformations
===========================

Introduces transformations and demonstrates their use on a toy corpus.



In this tutorial, I will show how to transform documents from one vector representation
into another. This process serves two goals:

1. To bring out hidden structure in the corpus, discover relationships between
   words and use them to describe the documents in a new and
   (hopefully) more semantic way.
2. To make the document representation more compact. This both improves efficiency
   (new representation consumes less resources) and efficacy (marginal data
   trends are ignored, noise-reduction).

Creating the Corpus
-------------------

First, we need to create a corpus to work with.
This step is the same as in the previous tutorial;
if you completed it, feel free to skip to the next section.



### Creating a transformation


The transformations are standard Python objects, typically initialized by means of training corpus:




In [ ]:
from gensim import models

tfidf = models.TfidfModel(bow_corpus)  # step 1 -- initialize a model

We used our old corpus from tutorial 1 to initialize (train) the transformation model. Different
transformations may require different initialization parameters; in case of TfIdf, the
"training" consists simply of going through the supplied corpus once and computing document frequencies
of all its features. Training other models, such as Latent Semantic Analysis or Latent Dirichlet
Allocation, is much more involved and, consequently, takes much more time.

<div class="alert alert-info"><h4>Note</h4><p>Transformations always convert between two specific vector
  spaces. The same vector space (= the same set of feature ids) must be used for training
  as well as for subsequent vector transformations. Failure to use the same input
  feature space, such as applying a different string preprocessing, using different
  feature ids, or using bag-of-words input vectors where TfIdf vectors are expected, will
  result in feature mismatch during transformation calls and consequently in either
  garbage output and/or runtime exceptions.</p></div>


### Transforming vectors


From now on, ``tfidf`` is treated as a read-only object that can be used to convert
any vector from the old representation (bag-of-words integer counts) to the new representation
(TfIdf real-valued weights):



In [ ]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

Or to apply a transformation to a whole corpus:



In [ ]:
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    print(doc)

In this particular case, we are transforming the same corpus that we used
for training, but this is only incidental. Once the transformation model has been initialized,
it can be used on any vectors (provided they come from the same vector space, of course),
even if they were not used in the training corpus at all. 
<div class="alert alert-info"><h4>Note</h4><p>Calling <code>model[corpus]</code> only creates a wrapper around the old <code>corpus</code> document stream -- actual conversions are done on-the-fly, during document iteration.
  We cannot convert the entire corpus at the time of calling <code>corpus_transformed = model[corpus]</code>,
  because that would mean storing the result in main memory, and that contradicts gensim's objective of memory-indepedence.
  If you will be iterating over the transformed <code>corpus_transformed</code> multiple times, and the
  transformation is costly, serialize the resulting corpus to disk first <corpus-formats> and continue
  using that.</p></div>

Transformations can also be serialized, one on top of another, in a sort of chain:



In [ ]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

Here we transformed our Tf-Idf corpus via [Latent Semantic Indexing](http://en.wikipedia.org/wiki/Latent_semantic_indexing>)
into a latent 2-D space (2-D because we set ``num_topics=2``). Now you're probably wondering: what do these two latent
dimensions stand for? Let's inspect with <code>models.LsiModel.print_topics</code>:



In [ ]:
lsi_model.print_topics(2)

(the topics are printed to log -- see the note at the top of this page about activating
logging)

It appears that according to LSI, "trees", "graph" and "minors" are all related
words (and contribute the most to the direction of the first topic), while the
second topic practically concerns itself with all the other words. As expected,
the first five documents are more strongly related to the second topic while the
remaining four documents to the first topic:



In [ ]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, text_corpus):
    print(doc, as_text)

Model persistency is achieved with the `save` and `load` functions:



In [ ]:
model_path = 'file/model-lsi.lsi'
lsi_model.save(model_path)  # same for tfidf, lda, ...
loaded_lsi_model = models.LsiModel.load(model_path)



Available transformations
--------------------------

Gensim implements several popular Vector Space Model algorithms:

* [Term Frequency * Inverse Document Frequency, Tf-Idf](http://en.wikipedia.org/wiki/Tf%E2%80%93idf>)
  expects a bag-of-words (integer values) training corpus during initialization.
  During transformation, it will take a vector and return another vector of the
  same dimensionality, except that features which were rare in the training corpus
  will have their value increased.
  It therefore converts integer-valued vectors into real-valued ones, while leaving
  the number of dimensions intact. It can also optionally normalize the resulting
  vectors to (Euclidean) unit length.
<code>model = models.TfidfModel(corpus, normalize=True)</code>

* [Latent Semantic Indexing, LSI (or sometimes LSA)](http://en.wikipedia.org/wiki/Latent_semantic_indexing>)
  transforms documents from either bag-of-words or (preferrably) TfIdf-weighted space into
  a latent space of a lower dimensionality. For the toy corpus above we used only
  2 latent dimensions, but on real corpora, target dimensionality of 200--500 is recommended
  as a "golden standard" [1]_.
<code>model = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=300)</code>

* [Random Projections, RP](http://www.cis.hut.fi/ella/publications/randproj_kdd.pdf) aim to
  reduce vector space dimensionality. This is a very efficient (both memory- and
  CPU-friendly) approach to approximating TfIdf distances between documents, by throwing in a little randomness.
  Recommended target dimensionality is again in the hundreds/thousands, depending on your dataset.
  <code>model = models.RpModel(tfidf_corpus, num_topics=500)</code>

* [Latent Dirichlet Allocation, LDA](http://en.wikipedia.org/wiki/Latent_Dirichlet_allocation>)
  is yet another transformation from bag-of-words counts into a topic space of lower
  dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA),
  so LDA's topics can be interpreted as probability distributions over words. These distributions are,
  just like with LSA, inferred automatically from a training corpus. Documents
  are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).
    <code>model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)</code>

* [Hierarchical Dirichlet Process, HDP](http://jmlr.csail.mit.edu/proceedings/papers/v15/wang11a/wang11a.pdf>)
  is a non-parametric bayesian method (note the missing number of requested topics):
<code>model = models.HdpModel(corpus, id2word=dictionary)</code>




Similarity Queries
==================

Demonstrates querying a corpus for similar documents.



Creating the Corpus
-------------------

First, we need to create a corpus to work with.
This step is the same as in the previous tutorial;
if you completed it, feel free to skip to the next section.



Similarity interface
--------------------

In the previous tutorials, we covered what it means to create a corpus in the Vector Space Model and how
to transform it between different vector spaces. A common reason for such a
charade is that we want to determine **similarity between pairs of
documents**, or the **similarity between a specific document and a set of
other documents** (such as a user query vs. indexed documents).

We first use this tiny corpus to define a 2-dimensional
LSI space:



In [ ]:
from gensim import models
lsi = models.LsiModel(bow_corpus, id2word=dictionary, num_topics=2)

For the purposes of this tutorial, there are only two things you need to know about LSI.
First, it's just another transformation: it transforms vectors from one space to another.
Second, the benefit of LSI is that enables identifying patterns and relationships between terms (in our case, words in a document) and topics.
Our LSI space is two-dimensional (`num_topics = 2`) so there are two topics, but this is arbitrary.
If you're interested, you can read more about LSI here: [Latent Semantic Indexing](https://en.wikipedia.org/wiki/Latent_semantic_indexing>):

Now suppose a user typed in the query `"Human computer interaction"`. We would
like to sort our nine corpus documents in decreasing order of relevance to this query.
Unlike modern search engines, here we only concentrate on a single aspect of possible
similarities---on apparent semantic relatedness of their texts (words). No hyperlinks,
no random-walk static ranks, just a semantic extension over the boolean keyword match:



In [ ]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

In addition, we will be considering [cosine similarity](http://en.wikipedia.org/wiki/Cosine_similarity>)
in Vector Space Modeling, but wherever the vectors represent probability distributions,
[different similarity measures](http://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Symmetrised_divergence>)
may be more appropriate.

#### Initializing query structures


To prepare for similarity queries, we need to enter all documents which we want
to compare against subsequent queries. In our case, they are the same nine documents
used for training LSI, converted to 2-D LSA space. But that's only incidental, we
might also be indexing a different corpus altogether.



In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[bow_corpus])  # transform corpus to LSI space and index it


Index persistency is handled via the standard `save` and `load` functions:



In [ ]:
index.save('file/deerwester.index')
index = similarities.MatrixSimilarity.load('file/deerwester.index')


#### Performing querie
To obtain similarities of our query document against the nine indexed documents:



In [ ]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

Cosine measure returns similarities in the range `<-1, 1>` (the greater, the more similar),
so that the first document has a score of 0.99809301 etc.

With some standard Python magic we sort these similarities into descending
order, and obtain the final answer to the query `"Human computer interaction"`:



In [ ]:
sims = sorted(enumerate(sims), key=lambda item: item[1], reverse=True)
for doc_position, doc_score in sims:
    print(doc_score, text_corpus[doc_position])

The thing to note here is that documents no. 2 (``"The EPS user interface management system"``)
and 4 (``"Relation of user perceived response time to error measurement"``) would never be returned by
a standard boolean fulltext search, because they do not share any common words with ``"Human computer interaction"``. However, after applying LSI, we can observe that both of
them received quite high similarity scores (no. 2 is actually the most similar!),
which corresponds better to our intuition of
them sharing a "computer-human" related topic with the query. In fact, this semantic
generalization is the reason why we apply transformations and do topic modelling
in the first place.



Word2Vec Model
==============

Introduces Gensim's Word2Vec model and demonstrates its use on the [Lee Evaluation Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf>)



Word2Vec is a widely used algorithm based on neural
networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow).
Using large amounts of unannotated plain text, word2vec learns relationships
between words automatically. The output are vectors, one vector per word.


This tutorial:

* Introduces ``Word2Vec`` as an improvement over traditional bag-of-words
* Shows off a demo of ``Word2Vec`` using a pre-trained model
* Demonstrates training a new model from your own data
* Demonstrates loading and saving models
* Introduces several training parameters and demonstrates their effect
* Discusses memory requirements
* Visualizes Word2Vec embeddings by applying dimensionality reduction

Review: Bag-of-words
--------------------
The bag-of-words model transforms each document to a fixed-length vector of integers.
For example, given the sentences:

- ``John likes to watch movies. Mary likes movies too.``
- ``John also likes to watch football games. Mary hates football.``

The model outputs the vectors:

- ``[1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]``
- ``[1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]``

Each vector has 10 elements, where each element counts the number of times a
particular word occurred in the document.
The order of elements is arbitrary.
In the example above, the order of the elements corresponds to the words:
``["John", "likes", "to", "watch", "movies", "Mary", "too", "also", "football", "games", "hates"]``.

Bag-of-words models are surprisingly effective, but have several weaknesses.

First, they lose all information about word order.

Second, the model does not attempt to learn the meaning of the underlying
words, and as a consequence, the distance between vectors doesn't always
reflect the difference in meaning.  The ``Word2Vec`` model addresses this
second problem.

Introducing: the ``Word2Vec`` Model
-----------------------------------

``Word2Vec`` is a more recent model that embeds words in a lower-dimensional
vector space using a shallow neural network. The result is a set of
word-vectors where vectors close together in vector space have similar
meanings based on context, and word-vectors distant to each other have
differing meanings. For example, ``strong`` and ``powerful`` would be close
together and ``strong`` and ``Paris`` would be relatively far.

The are two versions of this model and :py:class:`~gensim.models.word2vec.Word2Vec`
class implements them both:

1. Skip-grams (SG)
2. Continuous-bag-of-words (CBOW)


The Word2Vec Skip-gram model, for example, takes in pairs (word1, word2) generated by moving a
window across text data, and trains a 1-hidden-layer neural network based on
the synthetic task of given an input word, giving us a predicted probability
distribution of nearby words to the input. A virtual one-hot encoding of words goes through a 'projection layer' to the hidden layer; these projection
weights are later interpreted as the word embeddings. So if the hidden layer
has 300 neurons, this network will give us 300-dimensional word embeddings.

Continuous-bag-of-words Word2vec is very similar to the skip-gram model. It
is also a 1-hidden-layer neural network. The synthetic training task now uses
the average of multiple input context words, rather than a single word as in
skip-gram, to predict the center word. Again, the projection weights that
turn one-hot words into averageable vectors, of the same width as the hidden
layer, are interpreted as the word embeddings.


Word2Vec Demo
-------------

To see what ``Word2Vec`` can do, let's download a pre-trained model and play
around with it. We will fetch the Word2Vec model trained on part of the
Google News dataset, covering approximately 3 million words and phrases. Such
a model can take hours to train, but since it's already available,
downloading and loading it with Gensim takes minutes.

The model is approximately 2GB, so you'll need a decent network connection
to proceed.  Otherwise, skip ahead to the "Training Your Own Model" section
 below.


In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

A common operation is to retrieve the vocabulary of a model. That is trivial:



In [ ]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

We can easily obtain vectors for terms the model is familiar with:




In [ ]:
vec_king = wv['king']

In [ ]:
len(vec_king)

Unfortunately, the model is unable to infer vectors for unfamiliar words.
This is one limitation of Word2Vec: if this limitation matters to you, check
out the FastText model.




In [ ]:
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

Moving on, ``Word2Vec`` supports several word similarity tasks out of the
box.  You can see how the similarity intuitively decreases as the words get
less and less similar.




In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

Print the 5 most similar words to "car" or "minivan"



In [ ]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

Which of the below does not belong in the sequence?



In [ ]:
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

Training Your Own Model
-----------------------

To start, you'll need some data for training the model. For the following
examples, we'll use the [Lee Evaluation Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor), which you already have if you’ve installed Gensim.


This corpus is small enough to fit entirely in memory, but we'll implement a
memory-friendly iterator that reads it line-by-line to demonstrate how you
would handle a larger corpus.




In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

If we wanted to do any custom preprocessing, e.g. decode a non-standard
encoding, lowercase, remove numbers, extract named entities... All of this can
be done inside the ``MyCorpus`` iterator and ``word2vec`` doesn’t need to
know. All that is required is that the input yields one sentence (list of
utf8 words) after another.

Let's go ahead and train a model on our corpus.  Don't worry about the
training parameters much for now, we'll revisit them later.




In [ ]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

Once we have our model, we can use it in the same way as in the demo above.

The main part of the model is ``model.wv`` , where "wv" stands for "word vectors".




In [ ]:
vec_king = model.wv['king']

In [ ]:
len(vec_king)

Retrieving the vocabulary works the same way:



In [ ]:
for index, word in enumerate(model.wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(model.wv.index_to_key)} is {word}")

Storing and loading models
--------------------------

You'll notice that training non-trivial models can take time.  Once you've
trained your model and it works as expected, you can save it to disk.  That
way, you don't have to spend time training it all over again later.

You can store/load models using the standard gensim methods:




In [ ]:
model_path = 'file/gensim-model-word2vec'

model.save(model_path)

# The model is now safely stored in the path.
# You can copy it to other machines, share it with others, etc.
# To load a saved model:

new_model = gensim.models.Word2Vec.load(model_path)


In addition, you can load models created by the original C tool, both using
its text and binary formats:

 <code>model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)</code>

using gzipped/bz2 input works too, no need to unzip

 <code>model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)</code>




Training Parameters
-------------------

``Word2Vec`` accepts several parameters that affect both training speed and quality.

min_count
---------

``min_count`` is for pruning the internal dictionary. Words that appear only
once or twice in a billion-word corpus are probably uninteresting typos and
garbage. In addition, there’s not enough data to make any meaningful training
on those words, so it’s best to ignore them:

default value of min_count=5



In [ ]:
model = gensim.models.Word2Vec(sentences, min_count=10)

vector_size
-----------

``vector_size`` is the number of dimensions (N) of the N-dimensional space that
gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more
accurate) models. Reasonable values are in the tens to hundreds.




In [ ]:
# The default value of vector_size is 100.
model = gensim.models.Word2Vec(sentences, vector_size=200)

workers
-------

``workers`` , the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vecor))
is for training parallelization, to speed up training:




In [ ]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

The ``workers`` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use
one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and ``word2vec``
training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/ ).




Memory
------

At its core, ``word2vec`` model parameters are stored as matrices (NumPy
arrays). Each array is **#vocabulary** (controlled by the ``min_count`` parameter)
times **vector size** (the ``vector_size`` parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number
to two, or even one). So if your input contains 100,000 unique words, and you
asked for layer ``vector_size=200``\ , the model will require approx.
``100,000*200*4*3 bytes = ~229MB``.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would
take a few megabytes), but unless your words are extremely loooong strings, memory
footprint will be dominated by the three matrices above.




Evaluating
----------

``Word2Vec`` training is an unsupervised task, there’s no good way to
objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic
test examples, following the “A is to B as C is to D” task. It is provided in
the 'datasets' folder.

For example a syntactic analogy of comparative type is ``bad:worse;good:?``.
There are total of 9 types of syntactic comparisons in the dataset like
plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as
capital cities (``Paris:France;Tokyo:?``) or family members
(``brother:sister;dad:?``).




Gensim supports the same evaluation set, in exactly the same format:




In [ ]:
# model.wv.evaluate_word_analogies(datapath('questions-words.txt'))

This ``evaluate_word_analogies`` method takes an [optional parameter](http://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.evaluate_word_analogies>)
``restrict_vocab`` which limits which test examples are to be considered.




In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset
specific to your business based on it. It contains word pairs together with
human-assigned similarity judgments. It measures the relatedness or
co-occurrence of two words. For example, 'coast' and 'shore' are very similar
as they appear in the same context. At the same time 'clothes' and 'closet'
are less similar because they are related but not interchangeable.




In [ ]:
#model.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))


  Good performance on Google's or WS-353 test set doesn’t mean word2vec will
  work well in your application, or vice versa. It’s always best to evaluate
  directly on your intended task. For an example of how to use word2vec in a
  classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre)




Online training / Resuming training
-----------------------------------

Advanced users can load a model and continue training it with more sentences
and new vocabulary words.




In [ ]:
model = gensim.models.Word2Vec.load(model_path)
more_sentences = [
    ['Advanced', 'users', 'can', 'load', 'a', 'model',
     'and', 'continue', 'training', 'it', 'with', 'more', 'sentences'],
]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.epochs)



You may need to tweak the ``total_words`` parameter to ``train()``,
depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C
tool, ``KeyedVectors.load_word2vec_format()``. You can still use them for
querying/similarity, but information vital for training (the vocab tree) is
missing there.




Training Loss Computation
-------------------------

The parameter ``compute_loss`` can be used to toggle computation of loss
while training the Word2Vec model. The computed loss is stored in the model
attribute ``running_training_loss`` and can be retrieved using the function
``get_latest_training_loss`` as follows :




In [ ]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(
    sentences,
    min_count=1,
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42,
)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

Visualising Word Embeddings
---------------------------

The word embeddings made by the model can be visualised by reducing
dimensionality of the words to 2 dimensions using tSNE.

Visualisations can be used to notice semantic and syntactic trends in the data.

Example:

* Semantic: words like cat, dog, cow, etc. have a tendency to lie close by
* Syntactic: words like run, running or cut, cutting lie close together.

Vector relations like vKing - vMan = vQueen - vWoman can also be noticed.


The model used for the visualisation is trained on a small corpus. Thus
some of the relations might not be so clear.




In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)


FastText Model
==============

Introduces Gensim's fastText model and demonstrates its use on the Lee Corpus.



Here, we'll learn to work with fastText library for training word-embedding
models, saving & loading them and performing similarity operations & vector
lookups analogous to Word2Vec.



### When to use fastText?


The main principle behind [fastText](<https://github.com/facebookresearch/fastText>) is that the
morphological structure of a word carries important information about the meaning of the word.
Such structure is not taken into account by traditional word embeddings like Word2Vec, which
train a unique word embedding for every individual word.
This is especially significant for morphologically rich languages (German, Turkish) in which a
single word can have a large number of morphological forms, each of which might occur rarely,
thus making it hard to train good word embeddings.


fastText attempts to solve this by treating each word as the aggregation of its subwords.
For the sake of simplicity and language-independence, subwords are taken to be the character ngrams
of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.


fastText can obtain vectors even for out-of-vocabulary (OOV) words, by summing up vectors for its
component char-ngrams, provided at least one of the char-ngrams was present in the training data.




### Training models




For the following examples, we'll use the Lee Corpus (which you already have if you've installed Gensim) for training our model.


In [ ]:
from pprint import pprint as print
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath

# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

model = FastText(vector_size=100)

# build the vocabulary
model.build_vocab(corpus_file=corpus_file)

# train the model
model.train(
    corpus_file=corpus_file, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words,
)

print(model)

### Training hyperparameters





Hyperparameters for training the model follow the same pattern as Word2Vec. FastText supports the following parameters from the original word2vec:

- model: Training architecture. Allowed values: `cbow`, `skipgram` (Default `cbow`)
- vector_size: Dimensionality of vector embeddings to be learnt (Default 100)
- alpha: Initial learning rate (Default 0.025)
- window: Context window size (Default 5)
- min_count: Ignore words with number of occurrences below this (Default 5)
- loss: Training objective. Allowed values: `ns`, `hs`, `softmax` (Default `ns`)
- sample: Threshold for downsampling higher-frequency words (Default 0.001)
- negative: Number of negative words to sample, for `ns` (Default 5)
- epochs: Number of epochs (Default 5)
- sorted_vocab: Sort vocab by descending frequency (Default 1)
- threads: Number of threads to use (Default 12)


In addition, fastText has three additional parameters:

- min_n: min length of char ngrams (Default 3)
- max_n: max length of char ngrams (Default 6)
- bucket: number of buckets used for hashing ngrams (Default 2000000)


Parameters ``min_n`` and ``max_n`` control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If ``max_n`` is set to 0, or to be lesser than ``min_n`` , no character ngrams are used, and the model effectively reduces to Word2Vec.



To bound the memory requirements of the model being trained, a hashing function is used that maps ngrams to integers in 1 to K. For hashing these character sequences, the [Fowler-Noll-Vo hashing function](http://www.isthe.com/chongo/tech/comp/fnv)(FNV-1a variant) is employed.




Saving/loading models
---------------------




Models can be saved and loaded via the ``load`` and ``save`` methods, just like
any other model in Gensim.




In [ ]:
# Save a model trained via Gensim's fastText implementation to temp.
import tempfile
import os
with tempfile.NamedTemporaryFile(prefix='saved_model_gensim-', delete=False) as tmp:
    model.save(tmp.name, separately=[])

# Load back the same model.
loaded_model = FastText.load(tmp.name)
print(loaded_model)

os.unlink(tmp.name)  # demonstration complete, don't need the temp file anymore

The ``save_word2vec_format`` is also available for fastText models, but will
cause all vectors for ngrams to be lost.
As a result, a model loaded in this way will behave as a regular word2vec model.




Word vector lookup
------------------


All information necessary for looking up fastText words (incl. OOV words) is
contained in its ``model.wv`` attribute.

If you don't need to continue training your model, you can export & save this `.wv`
attribute and discard `model`, to save space and RAM.




In [ ]:
wv = model.wv
print(wv)

# FastText models support vector lookups for out-of-vocabulary words by 
# summing up character ngrams belonging to the word.

print('night' in wv.key_to_index)

In [ ]:
print('nights' in wv.key_to_index)

In [ ]:
print(wv['night'])

In [ ]:
print(wv['nights'])

Similarity operations
---------------------




Similarity operations work the same way as word2vec. **Out-of-vocabulary words can also be used, provided they have at least one character ngram present in the training data.**




In [ ]:
print("nights" in wv.key_to_index)

In [ ]:
print("night" in wv.key_to_index)

In [ ]:
print(wv.similarity("night", "nights"))

Syntactically similar words generally have high similarity in fastText models, since a large number of the component char-ngrams will be the same. As a result, fastText generally does better at syntactic tasks than Word2Vec. 


#### Other similarity operations

The example training corpus is a toy corpus, results are not expected to be good, for proof-of-concept only



In [ ]:
print(wv.most_similar("nights"))

In [ ]:
print(wv.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant']))

In [ ]:
print(wv.doesnt_match("breakfast cereal dinner lunch".split()))

In [ ]:
print(wv.most_similar(positive=['baghdad', 'england'], negative=['london']))

In [ ]:
#print(wv.evaluate_word_analogies(datapath('questions-words.txt')))